In [1]:
from bs4 import BeautifulSoup
import requests
import os
# https://data.binance.vision/?prefix=data/spot/monthly/klines/
# https://github.com/binance/binance-public-data

In [6]:
url = "https://data.binance.vision/?prefix=data/spot/monthly/klines/1INCHBTC/1m/"

with open("/home/ema/dev/shocks/data/binance_pairs.html") as file:
    soup = BeautifulSoup(file)

In [7]:
pairs = []
for pair in soup.find_all("a")[1: ]:
    pairs.append(pair.text.replace("/", ""))

def download_data(data_path, pair, months, freq="1m"):
    for month in months:
        pair_dir = os.path.join(DATA_PATH, pair)
        if not os.path.isdir(pair_dir):
            os.mkdir(pair_dir)

        url = f"https://data.binance.vision/data/spot/monthly/klines/{pair}/{freq}/{pair}-{freq}-{month}.zip"
        os.system(f" cd {pair_dir} && wget {url}")
 

In [ ]:
months = [
    "2021-10",
    "2021-11",
    "2021-12",
    "2022-01",
    "2022-02",
    "2022-03",
]

freq = "1m"
DATA_PATH ="/home/ema/dev/shocks/shocks/data/binance/"
selected_pairs = ("BTCUSDT", "BNBBTC", "AVAXBTC", "JOEBTC", "AXSTRY", "CAKEBRL")


for pair in selected_pairs:
    download_data(data_path=DATA_PATH, pair=pair, months=months, freq="1m")

In [2]:
%load_ext autoreload
%autoreload 2

In [7]:
from shocks.dataset import Dataset
import pandas as pd
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams["figure.figsize"] = (20,15)

data = Dataset(pair="AVAXBTC")
data.preprocess(freq="5m")
#data.fit(window=250)

In [73]:
fit = data.fitted.copy()

In [35]:
len(data.shocks), len(data.data)

(25567, 4368)

In [8]:
%%time
shock_features = data.build_dataset()
# 30 it/sec


shocks detected. Time:  0.16661381721496582
Data fitted. Time:  1821.0170781612396


  0%|          | 0/358 [00:00<?, ?it/s]

/home/ema/dev/shocks/shocks/dataset.py:269: RuntimeWarning: divide by zero encountered in true_divide
  }
/home/ema/dev/shocks/shocks/dataset.py:268: RuntimeWarning: divide by zero encountered in true_divide
  "tot_pct_change": lambda x: 100 * (x[:, -1] - x[:, 0]) / x[:, 0],
/home/ema/dev/shocks/shocks/dataset.py:268: RuntimeWarning: invalid value encountered in true_divide
  "tot_pct_change": lambda x: 100 * (x[:, -1] - x[:, 0]) / x[:, 0],
/home/ema/dev/shocks/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:232: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


Shocks processed. Time:  1.2947778701782227


  0%|          | 0/17900 [00:00<?, ?it/s]

/home/ema/dev/shocks/shocks/dataset.py:269: RuntimeWarning: invalid value encountered in true_divide
  }
/home/ema/dev/shocks/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:179: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/home/ema/dev/shocks/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:212: RuntimeWarning: invalid value encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)


Non shocks processed. Time:  20.739856004714966
AVAXBTC has been processed succesfully
CPU times: user 1h 19min 27s, sys: 1min 17s, total: 1h 20min 44s
Wall time: 30min 48s


In [6]:
data.find_shocks(start_date="2021-10", end_date="2022-03", plot=False, std_from_mean=3.5)
# data.fit(window=250, start_date="2021-10-01", end_date="2021-11-3")
# data.plot_fit(data.fitted, data.shocks)

In [12]:
100 * len(data.shocks)/len(data.data)

1.4194139194139195

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import pickle
with open('/home/ema/dev/shocks/data/processed/AVAXBTC_1h.pkl', 'rb') as f:
    d = pickle.load(f)

for el in d['features']:
     direction = el['direction']
     el['direction'] = 1 if direction == -1 else direction

df = pd.DataFrame.from_dict(d['features'])
cols = [col for col in df.columns if col != 'direction']
labels = df['direction'].tolist()
X_train, X_test, y_train, y_test = train_test_split(df[cols], labels, test_size=0.5, random_state=42, stratify=labels)

In [5]:
class_weights = {
   # -1: (len([el for el in labels if el == -1]) / len(labels)),
    0: (len([el for el in labels if el == 0]) / len(labels)),
    1: (len([el for el in labels if el == 1]) / len(labels)),
}

model = RandomForestClassifier(class_weight=class_weights)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print(classification_report(y_test, preds, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       775
           1       1.00      1.00      1.00        15

    accuracy                           1.00       790
   macro avg       1.00      1.00      1.00       790
weighted avg       1.00      1.00      1.00       790



In [6]:
class_weights

{0: 0.9810126582278481, 1: 0.0189873417721519}